In [11]:
import enviroments_package
import gymnasium

world_dir = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/simulation_package/worlds/my_frst_webots_world.wbt"
json_take_off = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/configs/reward_package_config/takeoff.json"

In [12]:
from enviroments_package import RemoveKeyObservationWrapper, ScaleRewardWrapper, ScaleActionWrapper
from stable_baselines3.common.env_util import make_vec_env

num_envs = 1

#def make_env():
env = gymnasium.make('drone_tfg_juanes/Drone-v1', simulation_path = world_dir, reward_json_path = json_take_off, no_render=False)

env = RemoveKeyObservationWrapper(env, remove_keys=["camera", "gps"])
env = ScaleRewardWrapper(env, scale_factor=0.1)
env = ScaleActionWrapper(env, in_low=0, in_high=2, out_low=0, out_high=576)
    #return env

#env = make_vec_env(make_env, n_envs=num_envs)

In [13]:
from stable_baselines3.common.callbacks import BaseCallback


class TrainingCallback(BaseCallback):
    def __init__(self, env, verbose=1):
        super(TrainingCallback, self).__init__(verbose)
        self.env = env


    def _on_step(self) -> bool:
        return True

    def _on_rollout_start(self) -> None:
        self.env.reset()

    def _on_training_end(self):
        # Este método se llama al final del entrenamiento
        print("Entrenamiento finalizado. Cerrando el entorno...")
        self.env.close()  # Cerrar el entorno para evitar problemas


# Usar el callback al entrenar

callback = TrainingCallback(env=env)

In [14]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.logger import configure

timesteps = 2048
log_dir = "./logs/ppo_training/"
os.makedirs(log_dir, exist_ok=True)

# Configurar el logger con salida en csv, log, stdout y tensorboard
new_logger = configure(log_dir, ["stdout", "csv", "log", "tensorboard"])

# Verificar si el archivo ppomodel existe en la carpeta ./models/
if not os.path.exists('./models/ppomodel'):
    # Crear y entrenar el modelo solo si el archivo no existe
    model = PPO(
        "MultiInputPolicy",
        env,
        verbose=1,
        n_steps=512,        # Tamaño típico, controla el buffer de experiencias para actualizar la política
        batch_size=32,      # Tamaño del lote, normalmente 64 o 128
        learning_rate=3e-4, # Tasa de aprendizaje típica en PPO
        ent_coef=0.05       # Coeficiente de entropía para exploración, ajuste común en PPO
    )
    model.set_logger(new_logger)  # Asignar el nuevo logger al modelo
    model.learn(total_timesteps=timesteps, callback=callback)
    model.save('./models/ppomodel')  # Guardar el modelo para futuras ejecuciones
else:
    # Cargar el modelo guardado
    model = PPO.load("./models/ppomodel", env=env)
    model.set_logger(new_logger)  # Asignar el nuevo logger al modelo cargado
    # Continuar el entrenamiento desde el último punto
    model.learn(total_timesteps=timesteps, callback=callback)  # Ajusta los timesteps si quieres más entrenamiento
    model.save(path="./models/ppomodel")

Logging to ./logs/ppo_training/
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.83     |
|    ep_rew_mean     | 3.62     |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 639      |
|    total_timesteps | 512      |
---------------------------------
WEBOTS Result: INFO: xyz_controller: Starting controller: python.exe -u xyz_controller.py
Simulation Starting
RESET 
RESET 
SET_ALL_MOTORS [ 0.      80.92249  0.       0.     ]
SET_ALL_MOTORS [  0.      234.82782   0.       73.09015]
SET_ALL_MOTORS [29.961695 51.605312  0.        0.      ]
SET_ALL_MOTORS [286.81134   0.       52.58417   0.     ]
SET_ALL_MOTORS [576.        0.       77.13182 393.26773]
SET_ALL_MOTORS [ 0.        0.        0.       18.982758]
SET_ALL_MOTORS [0. 0. 0. 0.]
SET_ALL_MOTORS [0. 0. 0. 0.]
SET_ALL_MOTOR

In [20]:
import os
import shutil
from datetime import datetime


def move_and_rename_csv(src_dir, dst_dir, new_name):
    # Buscar el archivo CSV en el directorio fuente
    csv_files = [f for f in os.listdir(src_dir) if f.endswith('.csv')]

    # Verificar si hay algún archivo CSV en el directorio de origen
    if not csv_files:
        print("No se encontró ningún archivo CSV en el directorio de origen.")
        return

    # Tomar el primer archivo CSV encontrado
    csv_file = csv_files[0]
    src_path = os.path.join(src_dir, csv_file)
    dst_path = os.path.join(dst_dir, new_name)

    # Mover y renombrar el archivo
    shutil.copy2(src_path, dst_path)
    print(f"Archivo copiado y renombrado a {dst_path}")


# Ejemplo de uso
src_directory = log_dir
dst_directory = './data_collected/'  # Cambia por la ruta de tu carpeta de destino
new_filename = f'ppo{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'  # Cambia por el nombre deseado del archivo en el destino

move_and_rename_csv(src_directory, dst_directory, new_filename)

Archivo copiado y renombrado a ./data_collected/ppo20241104_215904.csv


In [15]:
observation, _ = env.reset()

for i in range(100):  # Cambia el número de pasos según quieras observar
    action, _states = model.predict(observation)  # Usa deterministic=True para ver la política aprendida
    observation, reward, terminated, truncated, _ = env.step(action)

    if terminated:
        observation, _ = env.reset()  # Reiniciar el entorno al final del episodio

# Cerrar el entorno después de la prueba
env.close()